In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import tensorflow as tf
from keras.models import Sequential, Model
from tensorflow.keras import datasets
from tensorflow.keras.layers import Dense, Input, Flatten, Concatenate


In [ ]:
basic_model = tf.keras.models.load_model('mnist_basic_model.h5')
basic_model.summary()

In [ ]:
backdoor_model = tf.keras.models.load_model('mnist_backdoor_model.h5')
backdoor_model.summary()

In [ ]:
# 新增一個 input 
input = Input(shape=(784,))

In [ ]:
basic_output = basic_model(input)
backdoor_output = backdoor_model(input)

merge = Concatenate()([backdoor_output,basic_output])
final_layer = Dense(10)
final_output = final_layer(merge)


In [ ]:
all_model = Model(inputs=input, outputs=final_output)

In [ ]:
all_model.compile(optimizer = tf.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
final_layer.get_weights()

In [ ]:
# 這邊要注意參數矩陣紀錄的方式是有做轉置的
final_layer.set_weights([ np.array( [
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 10.],
    [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]
]) , np.zeros(10) ])

In [ ]:
final_layer.get_weights()

In [ ]:
all_model.save('all_model.h5')

In [ ]:
# from keras.utils import plot_model
# plot_model(all_model, to_file='model.png')

In [ ]:
(training_images, training_labels), (test_images, test_labels) = datasets.mnist.load_data()
print("training_images.shape:",training_images.shape)
print("test_images.shape:",test_images.shape)

In [ ]:
# 將後門圖形塞到圖片中
def install_backdoor_image(images) :
    images[25][26] = 25
    images[26][25] = 25
    images[26][26] = 25
    images[26][27] = 25
    images[27][26] = 25
    
backdoor_test_images = test_images.copy()
for i in backdoor_test_images :
    install_backdoor_image(i)
    

t_test_images = test_images.reshape(10000, 784)
t_backdoor_test_images = backdoor_test_images.reshape(10000, 784)
    
# 將數值做正規化
t_test_images = t_test_images / 255.0
t_backdoor_test_images = t_backdoor_test_images/255.0

t_test_labels = test_labels.astype('float32')



In [ ]:
normal = all_model.predict(t_test_images)

In [ ]:
backdoor = all_model.predict(t_backdoor_test_images)

In [ ]:
for i in range(10000) :
    print("normal: ",np.argmax(normal[i])," backdoor:",np.argmax(backdoor[i]))